# Pendulum

In [71]:
%load_ext autoreload
%autoreload 2

In [72]:
import sys
sys.path.append("..")
sys.path.append(".")
import torch
import numpy as np
import copy
import plotly
import plotly.graph_objs as go
import pickle
from datetime import datetime
plotly.offline.init_notebook_mode(connected=True)

import robust_value_approx.samples_generator as samples_generator
import robust_value_approx.samples_buffer as samples_buffer
import robust_value_approx.value_approximation as value_approximation
import robust_value_approx.training_log as training_log
import robust_value_approx.controllers as controllers

from pendulum_utils import get_value_function

In [74]:
offline_horizon = 10
vf, sys = get_value_function(offline_horizon)
V = vf.get_value_function()

In [19]:
x0 = torch.Tensor([np.pi+1.5, 1.]).double()
v, res = V(x0)
sys.plot_result(res).show()

In [23]:
x_nom = torch.tensor([np.pi, 0.], dtype=vf.dtype)
u_nom = torch.zeros(vf.u_dim[0], dtype=vf.dtype)
Q = torch.diag(torch.Tensor([1., 10.]))
R = torch.diag(torch.Tensor([1.]))
ctrl, S = controllers.get_lqr_controller(sys.dx, x_nom, u_nom, Q, R, vf.u_lo[0], vf.u_up[0])

ctrl = controllers.get_limited_lookahead_controller(get_value_function(offline_horizon)[0])

x0 = torch.tensor([np.pi+1.5, 0.], dtype=vf.dtype)
x_traj_sim, t_traj_sim = controllers.sim_ctrl(x0, vf.u_dim[0], sys.dx, ctrl, .1, 50)

fig = go.Figure()
for i in range(2):
    fig.add_trace(go.Scatter(
        x=t_traj_sim,
        y=x_traj_sim[i,:]
    ))
fig.show()

In [36]:
opt = dict(
    max_buffer_size = None,
    batch_size = 10,
    learning_rate = 1e-3,
    
    nn_width = 50,
    nn_depth = 1,

    num_samples_validation = 1000,
    
    init_num_samples = [100, 100],
    init_num_trainig_step = 1000,

#     num_generations = 100,
#     num_samples_per_generation = 10,
#     num_train_step_per_gen = 100,
    
#     adv_max_iter = 3,
#     adv_conv_tol = 1e-5,
#     adv_learning_rate = .25,
)

In [31]:
sys_name = 'pendulum'
validation_file = '../data/validation_' + sys_name
init_file = '../data/init_' + sys_name

In [187]:
x0_lo = torch.tensor([np.pi-1.5, -15.], dtype=vf.dtype)
x0_up = torch.tensor([np.pi+1.5, 15.], dtype=vf.dtype)

samples_gen_rand = samples_generator.RandomSampleGenerator(vf, x0_lo, x0_up)
samples_gen_grid = samples_generator.GridSampleGenerator(vf, x0_lo, x0_up)
samples_gen_adv = samples_generator.AdversarialSampleGenerator(vf, x0_lo, x0_up, 
                                                               max_iter=opt['adv_max_iter'],
                                                               conv_tol=opt['adv_conv_tol'],
                                                               learning_rate=opt['adv_learning_rate'])

KeyError: 'adv_max_iter'

In [48]:
x_validation, v_validation = samples_gen_rand.generate_samples(opt['num_samples_validation'],
                                                               include_time=True,
                                                               show_progress=True)

Progress: [########################################] 100.0%


In [54]:
torch.save(x_validation, validation_file + '_x.pt')
torch.save(v_validation, validation_file + '_v.pt')

In [55]:
x_validation = torch.load(validation_file + '_x.pt')
v_validation = torch.load(validation_file + '_v.pt')

In [59]:
x_init, v_init = samples_gen_grid.generate_samples(opt['init_num_samples'],
                                                   include_time=True,
                                                   show_progress=True)

Progress: [########################################] 100.0%


In [60]:
torch.save(x_init, init_file + '_x.pt')
torch.save(v_init, init_file + '_v.pt')

In [61]:
x_init = torch.load(init_file + '_x.pt')
v_init = torch.load(init_file + '_v.pt')

In [86]:
scaling = torch.mean(v_validation)

# model = value_approximation.QuadraticModel(vf.dtype,
#                                            1 + vf.x_dim[0],
#                                            scaling=scaling)

model = value_approximation.NeuralNetworkModel(vf.dtype,
                                               1 + vf.x_dim[0],
                                               opt['nn_width'],
                                               opt['nn_depth'],
                                               scaling=scaling)

vf_approx = value_approximation.ValueFunctionApproximation(model, learning_rate=opt['learning_rate'])

train_log = training_log.TrainingLog(prefix="baseline")

In [58]:
samples_buff = samples_buffer.SamplesBuffer(1 + vf.x_dim[0], 1, vf.dtype, max_size=opt['max_buffer_size'])

In [62]:
samples_buff.add_samples(x_init, v_init)

In [105]:
for train_step_i in range(opt['init_num_trainig_step']):
    x, v = samples_buff.get_random_samples(opt['batch_size'])
    loss = vf_approx.train_step(x, v)
    train_log.add_train_loss(loss)

In [110]:
x0 = torch.tensor([0., 15.], dtype=vf.dtype)
online_horizon = 3

In [111]:
ctrl = controllers.get_limited_lookahead_controller(get_value_function(online_horizon)[0], vf_approx)
x_traj_sim, t_traj_sim = controllers.sim_ctrl(x0, vf.u_dim[0], sys.dx, ctrl, .1, 50)
controllers.plot_sim(t_traj_sim, x_traj_sim, "Baseline").show()

In [112]:
ctrl = controllers.get_limited_lookahead_controller(get_value_function(online_horizon)[0])
x_traj_sim, t_traj_sim = controllers.sim_ctrl(x0, vf.u_dim[0], sys.dx, ctrl, .1, 50)
controllers.plot_sim(t_traj_sim, x_traj_sim, "Baseline - No lookahead").show()